In [84]:
# Get list of pipelines

from kfp import client
def get_pipeline_list():
    # Create a KFP client
    kfp_client = client.Client()
    
    # List all pipelines
    pipelines = kfp_client.list_pipelines()
    if pipelines is None or pipelines.pipelines is None:
        print("No pipeline found")
    else:   
        for pipeline in pipelines.pipelines:
            print(f"ID: {pipeline.pipeline_id} ,Name: {pipeline.display_name}")
            #print(pipeline)

###### Call the function to execute it           
get_pipeline_list()           

ID: 69f39504-90b4-4efa-9cc2-e236435c6f6e ,Name: chain_shipments_anonymize_chain
ID: c81c0821-a957-43b2-a1e1-430942a29702 ,Name: chain_shipments_data_chain


In [89]:
# This function will return the list of all runs.

from kfp import client

def get_pipeline_run_info():
    
    kfp_client = client.Client()

    runs = kfp_client.list_runs(page_size=5)
    if runs is None or runs.runs is None:
        print("No runs found")
    else:
        for run in runs.runs:
            print(f"Pipeline Run ID: {run.run_id},  Name: {run.display_name}, Status: {run.state}")

###### Call the function to execute it
get_pipeline_run_info()


Pipeline Run ID: 3fd14c99-0833-4888-b698-631ba6ba299c,  Name: chain_shipments_anonymize_chain_run, Status: SUCCEEDED
Pipeline Run ID: e7eddd8e-13fe-45bd-81ad-6e1f3a418ed4,  Name: chain_shipments_data_chain_run, Status: SUCCEEDED


In [54]:
#fetch the version info of existing pipelines


from kfp import client

class PipelineVersionInfo:
    def __init__(self, pipeline_name, version_name, version_pipeline_id):
        self.pipeline_name = pipeline_name
        self.version_name = version_name
        self.version_pipeline_id = version_pipeline_id

    def __repr__(self):
        return f"Pipeline Name: {self.pipeline_name}, Version Name: {self.version_name}, Version Pipeline ID: {self.version_pipeline_id}"

def fetch_pipelines_version_info():
    kfp_client = client.Client()
    
    pipelines = kfp_client.list_pipelines()
    result = []  # List to hold the PipelineVersionInfo objects
    
    if pipelines is None or pipelines.pipelines is None:
        print("No pipelines found")
        return []
    
    else:
        for pipeline in pipelines.pipelines:
            pipeline_versions = kfp_client.list_pipeline_versions(pipeline_id=pipeline.pipeline_id)
            for version in pipeline_versions.pipeline_versions:
                # Create a PipelineVersionInfo object and add it to the result list
                pipeline_info = PipelineVersionInfo(
                    pipeline_name=pipeline.display_name,
                    version_name=version.display_name,
                    version_pipeline_id=version.pipeline_version_id
                )
                result.append(pipeline_info)
    
    return result

##### Call the function to fetch versions info of the pipelines.
pipeline_info = fetch_pipelines_version_info()
print(pipeline_info)


No pipelines found
[]


In [55]:
#create a running experiment from a specific pipeline

def start_pipeline_experiment(pipeline_name:str,experiment_name:str,run_name:str,version_id:str):
    from kfp import client
    
    kfp_client = client.Client()
    #set up the Minio 
    MINIO_ENDPOINT = 'http://minio.kubeflow:9000'  # Update with your MinIO service endpoint
    BUCKET_NAME = 'my-bucket'  # Replace with your desired bucket name
    ACCESS_KEY = 'minio'  # Replace with your access key
    SECRET_KEY = 'minio123'  # Replace with your secret key
    DEFAULT_REGION ='us-east-1'
    arguments = {}
    
    
    pipelines = kfp_client.list_pipelines()
    pipeline_id = None
    for pipeline in pipelines.pipelines:
        if pipeline.display_name == pipeline_name:
            pipeline_id = pipeline.pipeline_id
            print(f"Pipeline ID: {pipeline_id}")
            break
    
    if pipeline_id is None:
        raise ValueError(f"Pipeline '{pipeline_name}' not found. Please check the pipeline name.")
    
    # Check if the experiment already exists
    experiments = kfp_client.list_experiments()
    experiment_id = None
    for experiment in experiments.experiments:
        if experiment.display_name == experiment_name:
            experiment_id = experiment.experiment_id
            print(f"Experiment ID: {experiment_id}")
            break
    
    # If the experiment doesn't exist, create a new one
    if experiment_id is None:
        experiment = kfp_client.create_experiment(name=experiment_name)
        experiment_id = experiment.experiment_id
        print(f"New Experiment ID: {experiment_id}")
        
    try:
        #Run the pipeline using the pipeline ID and experiment
        run = kfp_client.run_pipeline(
            experiment_id=experiment_id,
            job_name=run_name,
            pipeline_id=pipeline_id,
            params=arguments,
            version_id=version_id
        )
        
        print(f"Run submitted. Run ID: {run.run_id}")
        return {
            "pipeline_id": pipeline_id,
            "experiment_id": experiment_id,
            "run_id": run.run_id
        }
    except Exception as e:
        print(f"Error occurred for Pipeline '{pipeline_name}', Version ID '{version_id}': {e}")
        return None
    
    
##### Call the function to start an experiments for the pipeline.
start_pipeline_experiment("chain_shipments_anonymize_chain", "chain_shipments_anonymize_chain_exp","chain_shipments_anonymize_chain_run", "6a58e79b-0a5f-4757-bb2c-453288993103")


TypeError: 'NoneType' object is not iterable

In [79]:
# Delete all pipelines and its versions

from kfp import Client

def delete_pipelines():
    kfp_client = Client()
    
    pipelines = kfp_client.list_pipelines()
    
    for pipeline in pipelines.pipelines:
        # Get all versions of the pipeline and delete versions.
        versions = kfp_client.list_pipeline_versions(pipeline_id=pipeline.pipeline_id)
        
        for version in versions.pipeline_versions:
            kfp_client.delete_pipeline_version(pipeline.pipeline_id,version.pipeline_version_id)
            print(f"Deleted version: {version.display_name} (ID: {version.pipeline_version_id}) of pipeline: {pipeline.display_name}")
        
        # Now delete the pipeline itself
        kfp_client.delete_pipeline(pipeline.pipeline_id)
        print(f"Deleted pipeline: {pipeline.display_name} (ID: {pipeline.pipeline_id})")
        
##### Call the function to delete all the pipelines and their versions.
delete_pipelines()


Deleted version: chain_shipments_data_chain (ID: 5aa43f28-8a16-4dd7-9d15-bda2ebcfb151) of pipeline: chain_shipments_data_chain
Deleted pipeline: chain_shipments_data_chain (ID: 33c232a1-4e84-4ae0-a3ce-6f4200194192)
Deleted version: chain_shipments_anonymize_chain (ID: 0773dd77-c618-4af5-97cb-5cc8dd50849b) of pipeline: chain_shipments_anonymize_chain
Deleted pipeline: chain_shipments_anonymize_chain (ID: 4035e940-65b8-4a09-a8ea-a75113de4e73)


In [80]:
# Delete All the experiments that their runs which are in the state ['CANCELED', 'FAILED', 'SKIPPED','SUCCEEDED']

import kfp

def delete_completed_experiments():
    client = kfp.Client()
    
    # List all experiments
    experiments = client.list_experiments()
    
    # Iterate over all experiments
    if experiments is not None or experiments.experiments is not None:
        for experiment in experiments.experiments:
            print(f"Checking Experiment: {experiment.display_name} (ID: {experiment.experiment_id})")
            
            # List all runs for the current experiment
            runs = client.list_runs(experiment_id=experiment.experiment_id)
            
            all_runs_completed = True  # Flag to track if all runs are completed
            
            if runs and runs.runs:
                for run in runs.runs:
                    print(f"  Run Name: {run.display_name}, Run ID: {run.run_id}")
                    
                    # Check if the run status
                    if run.state not in ['CANCELED', 'FAILED', 'SKIPPED','SUCCEEDED','ERROR']:
                        print(f"  Run {run.display_name} is not completed. Skipping deletion of this experiment.")
                        all_runs_completed = False
                        break
            
            # If all runs are completed, delete the experiment and its runs
            if all_runs_completed:
                print(f"  All runs for experiment {experiment.display_name} are completed. Deleting runs and experiment.")
                
                # Delete all runs for this experiment
                if runs and runs.runs:
                    for run in runs.runs:
                        print(f"    Deleting run: {run.display_name}")
                        client.delete_run(run_id=run.run_id)
                    
                    # Delete the experiment
                    client.delete_experiment(experiment_id=experiment.experiment_id)
                    print(f"    Experiment {experiment.display_name} deleted.")
            else:
                print(f"  Skipping deletion of experiment {experiment.display_name} as some runs are still in progress.")
        
    print("Completed cleanup of experiments.")

##### Call the function to delete all the pipelines and their versions.    
delete_completed_experiments()   


Checking Experiment: Default (ID: b9a875a8-fb4f-4974-9be5-e1a547aaedf6)
  All runs for experiment Default are completed. Deleting runs and experiment.
Checking Experiment: pipeline_employee_data_experiment (ID: fa55e32b-423d-447b-b81f-fc8734bf21ee)
  All runs for experiment pipeline_employee_data_experiment are completed. Deleting runs and experiment.
Checking Experiment: chain_shipments_anonymize_chain_experiment (ID: 22e690e1-b383-4485-8c05-d11049427cd0)
  Run Name: chain_shipments_anonymize_chain_run, Run ID: a6779a02-4ce3-4e98-82a5-6ade8d31c4b4
  All runs for experiment chain_shipments_anonymize_chain_experiment are completed. Deleting runs and experiment.
    Deleting run: chain_shipments_anonymize_chain_run
    Experiment chain_shipments_anonymize_chain_experiment deleted.
Checking Experiment: chain_shipments_data_chain_experiment (ID: 3da6142d-e53d-4c43-82e8-a3fe9aeed053)
  Run Name: chain_shipments_data_chain_run, Run ID: b3d0302a-78ef-4279-bb55-46b6ec762fba
  All runs for expe

In [83]:
# main function for running a round of  experiments

def main ():
    
   # Deleting the experiments from the last round
   delete_completed_experiments()
   
   # Generating the new round of experiments for available pipelines 
   pipeline_version_info= fetch_pipelines_version_info() 
   if len(pipeline_version_info) == 0:
       print("No pipelines found")
   else:
      print(f"Pipeline version {len(pipeline_version_info)}")
      for pipeline_version in pipeline_version_info:
          print(pipeline_version)
          experiment_name = pipeline_version.pipeline_name + "_experiment"
          run_name = pipeline_version.pipeline_name + "_run"
          result = start_pipeline_experiment(pipeline_version.pipeline_name, experiment_name, run_name, pipeline_version.version_pipeline_id)
          if result:
            print(f"Pipeline ID: {result['pipeline_id']} , Experiment ID: {result['experiment_id']} , Run ID: {result['run_id']}")
          else:
            print(f"Failed to start pipeline for: {pipeline_version.pipeline_name}")
                 

main()

Checking Experiment: Default (ID: b9a875a8-fb4f-4974-9be5-e1a547aaedf6)
  All runs for experiment Default are completed. Deleting runs and experiment.
Checking Experiment: pipeline_employee_data_experiment (ID: fa55e32b-423d-447b-b81f-fc8734bf21ee)
  All runs for experiment pipeline_employee_data_experiment are completed. Deleting runs and experiment.
Completed cleanup of experiments.
Pipeline version 2
Pipeline Name: chain_shipments_anonymize_chain, Version Name: chain_shipments_anonymize_chain, Version Pipeline ID: 78e9afe7-0c79-4cae-94ad-4858d3c23805
Pipeline ID: 69f39504-90b4-4efa-9cc2-e236435c6f6e


New Experiment ID: 287a7584-66bb-4a10-8125-a788304be329


Run submitted. Run ID: 3fd14c99-0833-4888-b698-631ba6ba299c
Pipeline ID: 69f39504-90b4-4efa-9cc2-e236435c6f6e , Experiment ID: 287a7584-66bb-4a10-8125-a788304be329 , Run ID: 3fd14c99-0833-4888-b698-631ba6ba299c
Pipeline Name: chain_shipments_data_chain, Version Name: chain_shipments_data_chain, Version Pipeline ID: 7035cc69-70b8-4f7f-b5df-9cd88a2d6a01
Pipeline ID: c81c0821-a957-43b2-a1e1-430942a29702


New Experiment ID: 379766b9-9b03-435c-901c-8977ca4c784a


Run submitted. Run ID: e7eddd8e-13fe-45bd-81ad-6e1f3a418ed4
Pipeline ID: c81c0821-a957-43b2-a1e1-430942a29702 , Experiment ID: 379766b9-9b03-435c-901c-8977ca4c784a , Run ID: e7eddd8e-13fe-45bd-81ad-6e1f3a418ed4
